In [1]:
# < > No tengo estos simbolos en el teclado por lo que los tengo aquí para después....
import numpy as np
import pandas as pd # Manipular los datos de forma sencilla para poder estudiarlos previamente

import matplotlib.pyplot as plt # Representaciones
import seaborn as sns # Representaciones de correlaciones entre variables
%matplotlib inline

import scipy.stats as stats # Predicciones estadísticas

### Download data

In [2]:
df = pd.read_csv('C:/Users/Usuario/Downloads/all/train.csv', nrows=2)

In [3]:
new_df  = pd.DataFrame(columns = df.columns)

In [4]:
#df = pd.read_csv('C:/Users/Usuario/Downloads/all/train.csv', nrows=2)
#new_df = df.empty
chunksize = 100000
for chunk_number, df_train in enumerate(pd.read_csv('C:/Users/Usuario/Downloads/all/train.csv', chunksize=chunksize, nrows=1000000)):
    print(chunk_number*100000)
    new_df = new_df.select_dtypes('number').append(df_train)


C:\Users\Usuario\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
100000


C:\Users\Usuario\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


200000
300000
400000
500000
600000
700000
800000
900000


In [5]:
new_df.shape

(1000000, 83)

In [6]:
#df_train = new_df.select_dtypes('number').fillna(new_df.mean())
df_train = new_df.fillna(new_df.mean())

In [7]:
df_train.sample(5)

,AVProductStatesIdentifier,AVProductsEnabled,AVProductsInstalled,AppVersion,AutoSampleOptIn,AvSigVersion,Census_ActivationChannel,Census_ChassisTypeName,Census_DeviceFamily,Census_FirmwareManufacturerIdentifier,...,Processor,ProductName,PuaMode,RtpStateBitfield,SMode,SkuEdition,SmartScreen,UacLuaenable,Wdft_IsGamer,Wdft_RegionIdentifier
254247,24213.0,1.0,2.0,NaN,0,NaN,NaN,NaN,NaN,628.0,...,NaN,NaN,NaN,7.0,0.0,NaN,NaN,1.0,0.0,10.0
593142,49480.0,1.0,2.0,NaN,0,NaN,NaN,NaN,NaN,142.0,...,NaN,NaN,NaN,7.0,0.0,NaN,NaN,1.0,0.0,7.0
833486,53447.0,1.0,1.0,NaN,0,NaN,NaN,NaN,NaN,355.0,...,NaN,NaN,NaN,7.0,0.0,NaN,NaN,1.0,0.0,11.0
898370,7681.0,1.0,3.0,NaN,0,NaN,NaN,NaN,NaN,554.0,...,NaN,NaN,NaN,7.0,0.0,NaN,NaN,1.0,1.0,10.0
39557,53447.0,1.0,1.0,NaN,0,NaN,NaN,NaN,NaN,142.0,...,NaN,NaN,NaN,7.0,0.0,NaN,NaN,1.0,0.0,3.0


#### Quitamos todas las columnas que tengan mas del 10% de los datos NaN

In [8]:
temp = df_train.isna().sum() / df.shape[0] > 0.1
bad_columns = temp[temp == True].index

In [9]:
df = df_train.drop(bad_columns, axis=1)

In [10]:
df.dtypes

AVProductStatesIdentifier                            float64
AVProductsEnabled                                    float64
AVProductsInstalled                                  float64
AutoSampleOptIn                                        int64
Census_FirmwareManufacturerIdentifier                float64
Census_FirmwareVersionIdentifier                     float64
Census_HasOpticalDiskDrive                             int64
Census_InternalBatteryNumberOfCharges                float64
Census_InternalPrimaryDiagonalDisplaySizeInInches    float64
Census_InternalPrimaryDisplayResolutionHorizontal    float64
Census_InternalPrimaryDisplayResolutionVertical      float64
Census_IsAlwaysOnAlwaysConnectedCapable              float64
Census_IsFlightingInternal                           float64
Census_IsFlightsDisabled                             float64
Census_IsPenCapable                                    int64
Census_IsPortableOperatingSystem                       int64
Census_IsSecureBootEnabl

### Predictions

In [11]:
from sklearn.model_selection import train_test_split
# Separamos los datos en el conjunto de entrenamiento en el test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('HasDetections', axis=1), df["HasDetections"])

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = {
    
    
#    "KNN(3)"        : KNeighborsClassifier(3),
#    "Linear SVM"    : SVC(kernel="linear"),
#    "RBF SVM"       : SVC(gamma=2, C=1),
#    "Gaussian Proc" : GaussianProcessClassifier(1.0 * RBF(1.0)),
#    "Desition Tree" : DecisionTreeClassifier(max_depth=7),
#    "Random Forest" : RandomForestClassifier(max_depth=3, n_estimators=150),
    "Neural Net"    : MLPClassifier(alpha=1),
    "AdaBoost"      : AdaBoostClassifier(),
    "Naive Bayes"   : GaussianNB(),
    "QDA"           : QuadraticDiscriminantAnalysis()
}

In [14]:
for name, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    train = classifier.score(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print("Puntuacioes de {}, Entrenamiento: {:.2%}, Test: {:.2%}".format(name, train, score))

Puntuacioes de Neural Net, Entrenamiento: 49.97%, Test: 50.01%
Puntuacioes de AdaBoost, Entrenamiento: 60.15%, Test: 60.18%
Puntuacioes de Naive Bayes, Entrenamiento: 52.39%, Test: 52.38%
Puntuacioes de QDA, Entrenamiento: 56.67%, Test: 56.60%


Puntuacioes de KNN(3), Entrenamiento: 76.34%, Test: 53.95%
Puntuacioes de Neural Net, Entrenamiento: 49.97%, Test: 50.01%
Puntuacioes de AdaBoost, Entrenamiento: 60.15%, Test: 60.18%
Puntuacioes de Naive Bayes, Entrenamiento: 52.39%, Test: 52.38%
Puntuacioes de QDA, Entrenamiento: 56.67%, Test: 56.60%

In [15]:
from sklearn.metrics import roc_auc_score

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [16]:
X = df.drop('HasDetections', axis=1)
y = df['HasDetections']

Entrenamiento de Adaboost utilizando 1M de entradas y con GridSearchCV

In [17]:
from sklearn.ensemble import AdaBoostClassifier
accv = GridSearchCV(AdaBoostClassifier(), 
                   param_grid = {
                       'learning_rate':[0.01, 0.1],
                       'n_estimators':[70, 135, 200]
                   }, verbose=2)
accv.fit(X, y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] learning_rate=0.01, n_estimators=70 .............................
[CV] .............. learning_rate=0.01, n_estimators=70, total= 3.5min
[CV] learning_rate=0.01, n_estimators=70 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[CV] .............. learning_rate=0.01, n_estimators=70, total= 3.5min
[CV] learning_rate=0.01, n_estimators=70 .............................
[CV] .............. learning_rate=0.01, n_estimators=70, total= 3.5min
[CV] learning_rate=0.01, n_estimators=135 ............................
[CV] ............. learning_rate=0.01, n_estimators=135, total= 6.7min
[CV] learning_rate=0.01, n_estimators=135 ............................
[CV] ............. learning_rate=0.01, n_estimators=135, total= 6.7min
[CV] learning_rate=0.01, n_estimators=135 ............................
[CV] ............. learning_rate=0.01, n_estimators=135, total= 6.7min
[CV] learning_rate=0.01, n_estimators=200 ............................
[CV] ............. learning_rate=0.01, n_estimators=200, total= 9.9min
[CV] learning_rate=0.01, n_estimators=200 ............................
[CV] ............. learning_rate=0.01, n_estimators=200, total= 9.9min
[CV] learning_rate=0.01, n_estimators=200 ............................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 125.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.01, 0.1], 'n_estimators': [70, 135, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [18]:
accv.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=200, random_state=None)

Medición del acieto con la métrica propuesta: "Roc"

In [21]:
train_score = roc_auc_score(y_train, accv.predict_proba(X_train)[:, 1])
test_score = roc_auc_score(y_test, accv.predict_proba(X_test)[:, 1])
print("Train score = {:.2f} and Test score = {:.2f}.".format(train_score, test_score))

Train score = 0.63 and Test score = 0.63.


Entrenamiento de GradientBoostingClassifier utilizando 1M de entradas y con GridSearchCV

In [22]:
gbcv = GridSearchCV(GradientBoostingClassifier(), 
                   param_grid = {
                       'learning_rate':[0.1],
                       'n_estimators':[200],
                       'max_depth':[5]
                   }, verbose=2)
gbcv.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] learning_rate=0.1, max_depth=5, n_estimators=200 ................
[CV] . learning_rate=0.1, max_depth=5, n_estimators=200, total=23.8min
[CV] learning_rate=0.1, max_depth=5, n_estimators=200 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.8min remaining:    0.0s


[CV] . learning_rate=0.1, max_depth=5, n_estimators=200, total=23.5min
[CV] learning_rate=0.1, max_depth=5, n_estimators=200 ................
[CV] . learning_rate=0.1, max_depth=5, n_estimators=200, total=23.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 71.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.1], 'n_estimators': [200], 'max_depth': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [16]:
gbcv.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

Medición del acieto con la métrica propuesta: "Roc"

In [24]:
train_score = roc_auc_score(y_train, gbcv.predict_proba(X_train)[:, 1])
test_score = roc_auc_score(y_test, gbcv.predict_proba(X_test)[:, 1])
print("Train score = {:.2f} and Test score = {:.2f}.".format(train_score, test_score))

Train score = 0.67 and Test score = 0.67.


In [14]:
train_score

0.7128408317104356

In [15]:
test_score

0.6409982560592701

### Predict test

In [39]:
cols = df.drop('HasDetections', axis=1).columns

In [40]:
import os

In [41]:
submission_path = 'C:/Users/Usuario/Downloads/'
submission_file_name = 'submission.csv'

In [42]:
if submission_file_name in os.listdir(submission_path):
    print('ya existe ese archivo, cuidado')
else:
    print('continua')

continua


In [43]:
submission_file_path = submission_path + submission_file_name

In [44]:
with open(submission_file_path, 'w+') as file:
    file.write('MachineIdentifier,HasDetections\n')

In [45]:
nrows_test = 7853253
chunksize = 100000

In [46]:
for chunk_number, df_test in enumerate(pd.read_csv('C:/Users/Usuario/Downloads/all/test.csv', chunksize=chunksize)):
    print(f'{chunk_number * chunksize / nrows_test * 100: .1f}%')
    ids = df_test.MachineIdentifier
    df_test = df_test[cols]
    df_test = df_test.fillna(df_test.mean())
    
    preds = pd.DataFrame({'MachineIdentifier': ids, 'HasDetections': gbcv.predict_proba(df_test)[:, 1]})
    
    
    with open(submission_file_path, 'a') as file:
        preds.to_csv(file, index=False, header=False)

C:\Users\Usuario\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 0.0%
 1.3%
 2.5%
 3.8%
 5.1%
 6.4%
 7.6%
 8.9%
 10.2%
 11.5%
 12.7%
 14.0%
 15.3%
 16.6%
 17.8%
 19.1%
 20.4%
 21.6%
 22.9%
 24.2%
 25.5%
 26.7%
 28.0%
 29.3%
 30.6%
 31.8%
 33.1%
 34.4%
 35.7%
 36.9%
 38.2%
 39.5%
 40.7%
 42.0%
 43.3%
 44.6%
 45.8%
 47.1%
 48.4%


C:\Users\Usuario\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 49.7%
 50.9%
 52.2%
 53.5%
 54.8%
 56.0%
 57.3%
 58.6%
 59.8%
 61.1%
 62.4%
 63.7%
 64.9%
 66.2%
 67.5%
 68.8%
 70.0%
 71.3%
 72.6%
 73.9%
 75.1%
 76.4%
 77.7%
 78.9%
 80.2%
 81.5%
 82.8%
 84.0%
 85.3%
 86.6%
 87.9%
 89.1%
 90.4%
 91.7%
 93.0%
 94.2%
 95.5%
 96.8%
 98.0%
 99.3%


In [ ]:
# Second try. Same model trained with more examples.